In [ ]:
import requests
import polars as pl
import json
from box import Box
from IPython.display import display

In [ ]:
def get_size_of_encode():
    url = 'https://www.encodeproject.org/search/?type=Experiment&format=json&limit=all'
    headers = {'accept': 'application/json'}
    
    response = requests.get(url, headers=headers)
    data = response.json()
    experiment_graph = data.get('@graph', [])
    number_of_experiments = len(experiment_graph)
    number_of_files = 0
    
    for experiment in experiment_graph:
        experiment_files = experiment.get('files', [])
        experiment_file_count = len(experiment_files)
        number_of_files += experiment_file_count
    
    return number_of_experiments, number_of_files

In [ ]:
number_of_experiments, number_of_files = get_size_of_encode()
print(f"Total number of experiments on ENCODE: {number_of_experiments}")
print(f"Total number of files ENCODE: {number_of_files}")

In [ ]:
## Le broken ...

def all_files_to_static_json():
    url = 'https://www.encodeproject.org/files/?format=json&limit=all'
    headers = {'accept': 'application/json'}
    response = requests.get(url, headers=headers)
    data = response.json()
    file_graph = data.get('@graph', [])
    with open('encode.json', 'w') as f:
        json.dump(file_graph, f, indent=4)
        
    print('File graph saved to encode.json')

In [ ]:
all_files_to_static_json()

In [ ]:
import json
import polars as pl
from download-encode.constants import schema

with open('../encode.json') as f:
    data = json.load(f)
    
pl.from_dicts(data, strict=False, schema=schema).write_parquet('encode.parquet')

ModuleNotFoundError: No module named 'download_encode'

In [ ]:
import pickle

with open('../accessions.pkl', 'rb') as f:
    accessions = pickle.load(f)
    
print(len(accessions))

In [ ]:
encode = pl.read_parquet('../clean_encode.parquet')
display(encode)
# print(encode.select("biological_replicates"))
# display(encode)
for column in encode.iter_columns():
    display(encode.select(column).unique())
    
print(encode.select("output_type").unique().to_list())


In [ ]:
encode = pl.read_parquet('../clean_encode.parquet')

In [ ]:
display(encode)

In [ ]:
import json

json_objects = []

with open('../encode.jsonl') as f:
    for line in f:
        file = json.loads(line)
        json_objects.append(file)

json_string = json.dumps(json_objects)

from io import StringIO
json_file_obj = StringIO(json_string)

encode_json = pl.read_json(json_file_obj)

In [ ]:
import json 

url = "https://www.encodeproject.org/experiments/?format=json&limit=1"
response = requests.get(url)
data = response.json()
experiment = data['@graph'][0]
with open('encode_experiment_example.json', 'w') as outfile:
    json.dump(experiment, outfile, indent=4)


In [ ]:
"/experiments/ENCSR220XSM/".split('/')